In [ ]:
import torch
import umap
import pandas as pd
import numpy as np
import os
import sys
sys.path.append(os.getcwd()+"/../..")
from src import paths
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, precision_score, recall_score, ConfusionMatrixDisplay

In [ ]:
def plot_embeddings(embeddings: torch.tensor, labels:list, title = "plot", method="pca", aggregation="mean"):
    """
    Plot embeddings using PCA or UMAP
    """

    # Create a PCA object
    if method == "umap":
        reducer = umap.UMAP()
    elif method == "pca":
        reducer = PCA(n_components=2)
    elif method == "tsne":
        reducer = TSNE(n_components=2, perplexity=5, n_iter=250)
    else:
        raise ValueError("Reducer Method not implemented")

    # Aggregate embeddings along sequence dimension
    if aggregation == "mean":
        embeddings = torch.mean(embeddings, dim=1)
    elif aggregation == "cls":
        embeddings = embeddings[:,0,:]
    else:
        raise ValueError("Aggregation method not implemented")

    # Fit and transform the embeddings using the PCA object
    principalComponents = reducer.fit_transform(embeddings)

    # Create a dataframe with the embeddings and the corresponding labels
    df_embeddings = pd.DataFrame(principalComponents, columns=['x', 'y'])
    df_embeddings['label'] = labels
    
    for label in df_embeddings['label'].unique():
        _df = df_embeddings[df_embeddings['label'] == label]
        plt.scatter(_df['x'], _df['y'], alpha=0.5)
        plt.legend(df_embeddings['label'].unique())

    # Add a title and show the plot
    plt.title(title)

    plt.show()

In [ ]:
def summarise_line_label_performance(results_path: str):

    """
    Summarise the performance of a line labelling model on a given dataset split and plot the embeddings. Results dictionary
    should contain embeddings torch.tensor(n_test_samples, seq_length, n_classes), logits torch.tensor(n_test_samples, n_classes) and 
    true OHE labels (n_test_samples, n_classes).

    Args:
        results_path (str): Path to the results of classifier 1. Contains dictionary with embeddings, logits and true OHE labels.

    Returns:
        None 
    """

    # Load dataset
    dataset = torch.load(results_path)
    
    embeddings = dataset["embeddings"]
    logits = dataset["logits"]
    labels = dataset["labels"]
    print("Embeddings shape: ", embeddings.shape)
    print("Predictions shape: ", logits.shape)
    print("Labels shape: ", labels.shape)
    print("Labels example: ", labels[0])
    
    # Import label dict
    import json
    with open(os.path.join(paths.DATA_PATH_PREPROCESSED, "line_labelling/line_labelling_clean_class_mapping.json"), "r") as f:
        label_dict = json.load(f)
        
    # Accuracy
    preds = np.argmax(logits, axis=1)
    labels = np.argmax(labels, axis=1)
    
    # Map labels to class names
    label_names = [label_dict[str(label.item())] for label in labels]

    print(f"Accuracy: {torch.sum(preds == labels) / len(labels)}")
    
    # f1 score, precision, recall
    print(f"F1 Score: {f1_score(labels, preds, average='weighted')}")
    print(f"Precision: {precision_score(labels, preds, average='weighted')}")
    print(f"Recall: {recall_score(labels, preds, average='weighted')}")
    
    # Confusion Matrix
    plt.figure(figsize=(8, 6))
    ConfusionMatrixDisplay.from_predictions(labels, preds, xticks_rotation=90, display_labels=list(label_dict.values()))
    
    # Plot embeddings PCA
    plt.figure(figsize=(8, 6))
    plot_embeddings(embeddings=embeddings, labels=label_names, method="pca", aggregation="mean")
    
    # Plot embeddings UMAP
    plt.figure(figsize=(8, 6))
    plot_embeddings(embeddings=embeddings, labels=label_names, method="umap", aggregation="mean")

## Medbert Line Label Performance

In [ ]:
summarise_line_label_performance(paths.RESULTS_PATH/'line_labelling/medBERT-finetuned-test_output.pt')

## Bert Base Multilingual cased Performance

In [ ]:
summarise_line_label_performance(paths.RESULTS_PATH/'line_labelling/BERT-multilingual-finetuned-test_output.pt')